# LSTM

## PreProcessing

In [1]:
from google.colab import files
import shutil
import re
import random

import torch
import torchtext
from nltk.corpus import brown
from collections import Counter
import pandas as pd

In [2]:
# # preprocessing done on vs code, then uploaded cleaned corpus to colab

# # Function to clean a text column
# def clean_text(text):
#     # Remove periods, punctuation, quotes, and double quotes
#     text = re.sub(r'[.!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)
#     text = re.sub(r'[.,!"\'/\\]', '', text)


#     # Remove extra spaces
#     text = re.sub(r'\s+', ' ', text).strip()
#     return text

# def preprocess_csv(input_file, output_file):
#     # Read the CSV file
#     df = pd.read_csv(input_file)

#     # Remove the 'Class Index' column
#     df = df.drop(columns=['Class Index'])

#     # Apply the cleaning function to the 'Description' column
#     df['Description'] = df['Description'].apply(clean_text)

#     # Save the cleaned DataFrame to a new CSV file
#     df.to_csv(output_file, index=False)

# train_input_file = 'train.csv'
# test_input_file = 'test.csv'
# cleaned_train_output_file = 'cleanedtrain.csv'
# cleaned_test_output_file = 'cleanedtest.csv'

# # Preprocess the train and test CSV files
# preprocess_csv(train_input_file, cleaned_train_output_file)
# preprocess_csv(test_input_file, cleaned_test_output_file)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# uploaded = files.upload()

In [5]:

# # Path to the uploaded file in Colab (use the exact filename you see in the output)
# colab_file_path = "/content/cleanedtest.csv"

# # Destination directory in Google Drive
# drive_directory = "/content/drive/My Drive/"

# # Move the file to the specified directory
# shutil.move(colab_file_path, drive_directory)


In [3]:
#read input files
train_file = "/content/drive/My Drive/elmo_train.csv"
test_file = "/content/drive/My Drive/elmo_test.csv"

# Read the CSV files into DataFrames
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)


## Data

In [4]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import Vocab, build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import spacy
from collections import defaultdict

# Initialize the tokenizer
tokenizer = get_tokenizer('basic_english')


In [5]:
# Split the first 7600 entries of train_df into train and valid DataFrames
valid_df = train_df.iloc[:7600]
train_df = train_df.iloc[7600:]

In [6]:
# Tokenize the text data in the DataFrames
def tokenize_text(text):
    return tokenizer(text)

def create_vocab_manually(data, min_freq=1):
    vocab = {'<PAD>', '<UNK>'}
    word_freq = defaultdict(int)

    # Iterate over the tokenized data, building vocab and frequency counts
    for tokens in data:
        for token in tokens:
            word_freq[token] += 1

    # Create final vocab and frequency list
    rare_words = []
    for word, freq in word_freq.items():
        if freq < min_freq:
            rare_words.append(word)
        else:
            vocab.add(word)

    for word in rare_words:
        word_freq['<UNK>'] += word_freq[word]
        del word_freq[word]

    # Create a dictionary to map tokens to indices
    token2index = {token: idx for idx, token in enumerate(vocab)}

    return vocab, token2index



# Tokenize the text data in the DataFrames
train_data = train_df['Description'].apply(tokenize_text)

# Create vocabulary and token2index mapping
vocab, token2index = create_vocab_manually(train_data, min_freq=5)


In [7]:
# Define the maximum sentence length (you can adjust this value)
max_len = 100

# Function to pad sentences to a maximum length
def pad_sents(sents, pad_token):
    sents = [sent[:max_len] + [pad_token] * (max_len - len(sent[:max_len])) for sent in sents]
    return sents

# Function to convert tokens to indices using the token2index mapping
def tokens_to_indices(tokens):
    return [token2index.get(token, token2index['<UNK>']) for token in tokens]


In [111]:
# Convert tokens to indices for all datasets
train_data_indices = train_data.apply(tokens_to_indices)
valid_data_indices = valid_df['Description'].apply(tokenize_text).apply(tokens_to_indices)
test_data_indices = test_df['Description'].apply(tokenize_text).apply(tokens_to_indices)

# Pad the tokenized sentences
train_data_indices = pad_sents(train_data_indices, token2index['<PAD>'])
valid_data_indices = pad_sents(valid_data_indices, token2index['<PAD>'])
test_data_indices = pad_sents(test_data_indices, token2index['<PAD>'])

# Load labels from DataFrames
train_labels = train_df['Index'].values
valid_labels = valid_df['Index'].values
test_labels = test_df['Index'].values


In [112]:
class ELMoDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.LongTensor(self.data[idx]), torch.LongTensor([self.labels[idx]])

train_dataset = ELMoDataset(train_data_indices, train_labels)
valid_dataset = ELMoDataset(valid_data_indices, valid_labels)
test_dataset = ELMoDataset(test_data_indices, test_labels)


In [113]:
batch_size = 32

# Create DataLoader objects for training, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [94]:
print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))


3513
238
238


# ELMO MODEL

In [12]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [13]:
# Load pre-trained GloVe word vectors (You need to download the GloVe file)
glove_path = "/content/drive/My Drive/glove.6B.100d.txt"


In [14]:
print( type(vocab))

<class 'set'>


In [15]:
def load_glove_vectors(glove_path, token2index, embedding_dim=100):
    word_vectors = {}
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            if len(vector) != embedding_dim:
                # Handle dimension mismatch by truncating or zero-padding
                if len(vector) > embedding_dim:
                    vector = vector[:embedding_dim]
                else:
                    vector = np.pad(vector, (0, embedding_dim - len(vector)))

            word_vectors[word] = vector

    embedding_matrix = np.zeros((len(token2index), embedding_dim))
    for word, idx in token2index.items():
        if word in word_vectors:
            embedding_matrix[idx] = word_vectors[word]
        else:
            # Handle missing words (you can use random initialization or zeros)
            embedding_matrix[idx] = np.random.randn(embedding_dim)

    return embedding_matrix

embedding_matrix = load_glove_vectors(glove_path, token2index, embedding_dim=100)


In [16]:
class ELMo(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, batch_size, max_len, embedding_matrix):
        super(ELMo, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, input):
        embedded = self.embedding(input)
        lstm_output, _ = self.lstm(embedded)
        lstm_output = lstm_output.mean(dim=1)
        output = self.fc(lstm_output)
        return output


In [29]:
# Create an instance of the ELMo model
vocab_size = len(vocab)
embedding_dim = 100  # Make sure this matches the dimension of your GloVe vectors
hidden_dim = 128     # You can adjust this value
batch_size = 32
max_len = 100
elmo = ELMo(vocab_size, embedding_dim, hidden_dim, batch_size, max_len, embedding_matrix)

# Move the ELMo model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
elmo.to(device)


ELMo(
  (embedding): Embedding(25866, 100)
  (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [30]:
# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(elmo.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)  # Learning rate scheduler


In [31]:
# Training loop
num_epochs = 4  # You can adjust this value
best_valid_loss = float('inf')
for epoch in range(num_epochs):
    elmo.train()
    train_losses = []

    # Calculate the total number of batches for progress printing
    total_batches = len(train_loader)

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = elmo(inputs)

        loss = criterion(outputs, labels.float())
        train_losses.append(loss.item())

        loss.backward()
        optimizer.step()

        # Print progress
        progress = (batch_idx + 1) / total_batches * 100
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Batch [{batch_idx + 1}/{total_batches}] - Loss: {loss.item():.4f} - Progress: {progress:.2f}%")

    train_loss = np.mean(train_losses)

    elmo.eval()
    valid_losses = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = elmo(inputs)

            loss = criterion(outputs, labels.float())
            valid_losses.append(loss.item())

    valid_loss = np.mean(valid_losses)

    scheduler.step(valid_loss)  # Adjust learning rate based on validation loss

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(elmo.state_dict(), 'elmo_model.pt')  # Save the best model


Streaming output truncated to the last 5000 lines.
Epoch [3/4] - Batch [2027/3513] - Loss: -3978.8469 - Progress: 57.70%
Epoch [3/4] - Batch [2028/3513] - Loss: -3400.5010 - Progress: 57.73%
Epoch [3/4] - Batch [2029/3513] - Loss: -2677.2979 - Progress: 57.76%
Epoch [3/4] - Batch [2030/3513] - Loss: -4052.5820 - Progress: 57.79%
Epoch [3/4] - Batch [2031/3513] - Loss: -2967.4092 - Progress: 57.81%
Epoch [3/4] - Batch [2032/3513] - Loss: -3546.8091 - Progress: 57.84%
Epoch [3/4] - Batch [2033/3513] - Loss: -2750.8994 - Progress: 57.87%
Epoch [3/4] - Batch [2034/3513] - Loss: -4488.7974 - Progress: 57.90%
Epoch [3/4] - Batch [2035/3513] - Loss: -3982.4492 - Progress: 57.93%
Epoch [3/4] - Batch [2036/3513] - Loss: -2679.4067 - Progress: 57.96%
Epoch [3/4] - Batch [2037/3513] - Loss: -3621.2224 - Progress: 57.98%
Epoch [3/4] - Batch [2038/3513] - Loss: -3911.3569 - Progress: 58.01%
Epoch [3/4] - Batch [2039/3513] - Loss: -3259.8293 - Progress: 58.04%
Epoch [3/4] - Batch [2040/3513] - Loss:

In [22]:
model_path = "/content/drive/My Drive/elmo3.pt"

In [130]:
# Define the path to save the model in your Google Drive
model_path = "/content/drive/My Drive/elmo3.pt"
torch.save(elmo.state_dict(), model_path)

print(f"Model saved to Google Drive at: {model_path}")

Model saved to Google Drive at: /content/drive/My Drive/elmo2.pt


In [24]:
# import os

# # Specify the path to your model directory on Google Drive
# model_dir = '/content/drive/My Drive'

# # Change the current working directory to the model directory
# os.chdir(model_dir)

# # Load the best model
# elmo.load_state_dict(torch.load('elmo2.pt'))


<All keys matched successfully>

In [25]:
# elmo = torch.load('elmo2.pt')


In [34]:
# Evaluate the model on the test set
elmo.eval()
test_losses = []
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = elmo(inputs)

        loss = criterion(outputs, labels.float())
        test_losses.append(loss.item())

        predictions.extend(torch.sigmoid(outputs).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

test_loss = np.mean(test_losses)
predictions = np.array(predictions).flatten()
true_labels = np.array(true_labels).flatten()
# Ensure that true_labels and binary_predictions are binary (0 or 1)
true_labels = (true_labels > threshold).astype(int)
binary_predictions = (binary_predictions > threshold).astype(int)

threshold = 0.5  # You can adjust this threshold
binary_predictions = (predictions > threshold).astype(int)

accuracy = accuracy_score(true_labels, binary_predictions)
precision = precision_score(true_labels, binary_predictions)
recall = recall_score(true_labels, binary_predictions)
f1 = f1_score(true_labels, binary_predictions)

print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Test Loss: -5376.0562
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


## Text Classification model

In [35]:
# Import functions for calculating evaluation metrics
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix


In [36]:
# Print names, data, and shapes of trainable parameters in the elmo model
for name, param in elmo.named_parameters():
    if param.requires_grad:
        print(f"Name: {name}, Data: {param.data.shape}")


Name: lstm.weight_ih_l0, Data: torch.Size([512, 100])
Name: lstm.weight_hh_l0, Data: torch.Size([512, 128])
Name: lstm.bias_ih_l0, Data: torch.Size([512])
Name: lstm.bias_hh_l0, Data: torch.Size([512])
Name: lstm.weight_ih_l0_reverse, Data: torch.Size([512, 100])
Name: lstm.weight_hh_l0_reverse, Data: torch.Size([512, 128])
Name: lstm.bias_ih_l0_reverse, Data: torch.Size([512])
Name: lstm.bias_hh_l0_reverse, Data: torch.Size([512])
Name: fc.weight, Data: torch.Size([1, 256])
Name: fc.bias, Data: torch.Size([1])


In [37]:
# Extract and save ELMo embeddings to a NumPy array
elmo_embeddings = elmo.embedding.weight.data.cpu().numpy()
np.save('elmo_embeddings.npy', elmo_embeddings)


In [38]:
# Inspect the embedding of a specific word ("<UNK>")
word_idx = token2index['<UNK>']
print(f"Embedding for '<UNK>': {elmo_embeddings[word_idx]}")


Embedding for '<UNK>': [-1.0121417  -1.4156176  -0.0298482   0.6536329   0.95326215  1.0971441
 -0.56752086 -0.21581437 -1.6040913   0.62760323 -1.6215655   0.9224144
 -1.4988395   0.53647083 -2.907976   -0.4398041  -0.9194517   0.96832675
  0.10637516 -0.5483858   0.26476088 -0.01100634 -0.6127271   1.5152868
 -1.7216125  -0.92404264  0.09636644  0.27899024 -0.89142525  1.8279309
  1.0301975  -0.58982784 -1.4813957   1.5796335  -0.9929518   2.0217366
 -1.1030086  -0.5745664   1.5205176   0.31890973 -0.9007187  -0.24803726
  0.48897797 -0.6751636   0.69389856 -1.0621144  -0.37154797 -1.6198723
  1.4074097   0.3341644   0.08192572  0.98079085 -1.2001761  -0.920014
  0.4789514  -0.36087397  2.1498408   0.27008638  0.5320195  -0.7825608
  0.62592465  1.2411406  -0.32144362 -0.02751916 -0.87064666 -0.38262725
 -0.89374447 -0.971015    0.57736534 -1.0182662  -0.9285867   2.2005718
 -0.8894974   2.7497058  -0.25261205  0.5895088  -0.427491    0.47406447
  0.37252334 -0.87222844 -0.56585985 -

In [39]:
# Inspect the parameters of the LSTM layers in the elmo model
for name, param in elmo.lstm.named_parameters():
    if param.requires_grad:
        print(f"Name: {name}, Data: {param.data.shape}")


Name: weight_ih_l0, Data: torch.Size([512, 100])
Name: weight_hh_l0, Data: torch.Size([512, 128])
Name: bias_ih_l0, Data: torch.Size([512])
Name: bias_hh_l0, Data: torch.Size([512])
Name: weight_ih_l0_reverse, Data: torch.Size([512, 100])
Name: weight_hh_l0_reverse, Data: torch.Size([512, 128])
Name: bias_ih_l0_reverse, Data: torch.Size([512])
Name: bias_hh_l0_reverse, Data: torch.Size([512])


In [40]:
# Move ELMo embeddings to the specified device (GPU if available)
elmo_embeddings = torch.FloatTensor(elmo_embeddings).to(device)


In [142]:
max_len = 100  # Adjust as needed
batch_size = 32  # Adjust as needed
embedding_dim = 100  # Adjust as needed
hidden_dim = 128  # Adjust as needed
num_classes = 1  # Adjust as needed

# Define your custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.LongTensor(self.data[idx]), torch.LongTensor([self.labels[idx]])

# Load your data and create DataLoader objects
train_dataset = CustomDataset(train_data_indices, train_labels)
valid_dataset = CustomDataset(valid_data_indices, valid_labels)
test_dataset = CustomDataset(test_data_indices, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [143]:
class CustomTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(CustomTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input):
        embedded = self.embedding(input)
        lstm_output, _ = self.lstm(embedded)
        lstm_output = lstm_output.mean(dim=1)
        output = self.fc(lstm_output)
        return output

# Create an instance of your custom text classifier model
custom_model = CustomTextClassifier(vocab_size, embedding_dim, hidden_dim, num_classes)


In [144]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_model.to(device)

# Define your loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(custom_model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)  # Learning rate scheduler


In [145]:
num_epochs = 4  # Adjust as needed
best_valid_loss = float('inf')
for epoch in range(num_epochs):
    custom_model.train()
    train_losses = []

    total_batches = len(train_loader)

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = custom_model(inputs)

        loss = criterion(outputs, labels.float())
        train_losses.append(loss.item())

        loss.backward()
        optimizer.step()

        progress = (batch_idx + 1) / total_batches * 100
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Batch [{batch_idx + 1}/{total_batches}] - Loss: {loss.item():.4f} - Progress: {progress:.2f}%")

    train_loss = np.mean(train_losses)

    custom_model.eval()
    valid_losses = []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = custom_model(inputs)

            loss = criterion(outputs, labels.float())
            valid_losses.append(loss.item())

    valid_loss = np.mean(valid_losses)

    scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_model_state_dict = custom_model.state_dict()

Streaming output truncated to the last 5000 lines.
Epoch [3/4] - Batch [2027/3513] - Loss: -6537.3481 - Progress: 57.70%
Epoch [3/4] - Batch [2028/3513] - Loss: -5768.5923 - Progress: 57.73%
Epoch [3/4] - Batch [2029/3513] - Loss: -5384.3403 - Progress: 57.76%
Epoch [3/4] - Batch [2030/3513] - Loss: -6666.7197 - Progress: 57.79%
Epoch [3/4] - Batch [2031/3513] - Loss: -5769.6211 - Progress: 57.81%
Epoch [3/4] - Batch [2032/3513] - Loss: -5257.0801 - Progress: 57.84%
Epoch [3/4] - Batch [2033/3513] - Loss: -6411.4507 - Progress: 57.87%
Epoch [3/4] - Batch [2034/3513] - Loss: -6796.5415 - Progress: 57.90%
Epoch [3/4] - Batch [2035/3513] - Loss: -6796.9521 - Progress: 57.93%
Epoch [3/4] - Batch [2036/3513] - Loss: -6412.6104 - Progress: 57.96%
Epoch [3/4] - Batch [2037/3513] - Loss: -6284.7446 - Progress: 57.98%
Epoch [3/4] - Batch [2038/3513] - Loss: -6028.5000 - Progress: 58.01%
Epoch [3/4] - Batch [2039/3513] - Loss: -5259.3120 - Progress: 58.04%
Epoch [3/4] - Batch [2040/3513] - Loss:

In [ ]:
# Save the trained model
torch.save(custom_model.state_dict(), 'custom_model.pt')


In [146]:
# Define the path to save the model in your Google Drive
model_path = "/content/drive/My Drive/SenClass.pt"
torch.save(custom_model.state_dict(), model_path)

print(f"Model saved to Google Drive at: {model_path}")

Model saved to Google Drive at: /content/drive/My Drive/SenClass.pt


In [148]:
import os

# Specify the directory where you want to save the model
save_dir = '/content/drive/My Drive/'

# Ensure the directory exists; create it if necessary
os.makedirs(save_dir, exist_ok=True)

# Define the file name for the model
save_path = os.path.join(save_dir, 'custom_model.pt')

# Save the model
torch.save(custom_model.state_dict(), save_path)


In [152]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

In [150]:
# Testing the model on the test dataset
custom_model.eval()
test_preds = []
test_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = custom_model(inputs)
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Calculate test accuracy and F1 score
test_accuracy = accuracy_score(test_labels, test_preds)


In [ ]:
# Calculate test accuracy and F1 score
test_accuracy = accuracy_score(test_labels, test_preds)
test_f1_micro = f1_score(test_labels, test_preds, average='micro')
test_confusion_matrix = confusion_matrix(test_labels, test_preds)

print(f"Test Accuracy: {test_accuracy}")
print(f"Test F1 Micro: {test_f1_micro}")
print("Confusion Matrix:")
print(test_confusion_matrix)
